In [1]:
import numpy as np
import urllib
import re,sys

In [2]:
from pympler.asizeof import asizeof

In [3]:
a=[[1,0,0,1,0],[0,0,1,1,0]]
b = [1,0,1,1,0]
print('a: %s %s' %(sys.getsizeof(a) , asizeof(a)))
print('b: %s %s' %(sys.getsizeof(b) , asizeof(b)))

a: 80 344
b: 104 160


In [51]:
file_contents = []
with open('text.txt') as f:
        file_contents = f.read()

In [52]:
text = []
for val in file_contents.split('.'):
    sent = re.findall("[A-Za-z]+", val)
    text.append(' '.join(sent))

In [4]:
def get_inital_values(text):
    word_to_index= dict()
    index_to_word = dict()
    corpus = []
    count = 0
    vocab_size = 0
    
    for row in text:
        for word in row.split():
            word = word.lower()
            corpus.append(word)
            if word_to_index.get(word) == None:
                word_to_index.update ( {word : count})
                index_to_word.update ( {count : word })
                count  += 1
    vocab_size = len(word_to_index)
    length_of_corpus = len(corpus)
    
    return word_to_index,index_to_word,corpus,length_of_corpus,vocab_size

In [5]:
def get_one_hot_multiple_vectors(target_word,context_words):
    trgt_word_vector = np.zeros(vocab_size)
    
    index_of_word_dictionary = word_to_index.get(target_word) 
    trgt_word_vector[index_of_word_dictionary] = 1
    
    ctxt_word_vector = []
    
    
    for word in context_words:
        ctxt_word_vector_temp = np.zeros(vocab_size)
        index_of_word_dictionary = word_to_index.get(word) 
        ctxt_word_vector_temp[index_of_word_dictionary] = 1
        
        ctxt_word_vector.append(ctxt_word_vector_temp)
        
    
    return trgt_word_vector,ctxt_word_vector

In [6]:
def get_one_hot_single_vectors(target_word,context_words):
    trgt_word_vector = np.zeros(vocab_size)
    
    index_of_word_dictionary = word_to_index.get(target_word) 
    trgt_word_vector[index_of_word_dictionary] = 1
    
    ctxt_word_vector = np.zeros(vocab_size)
    
    
    for word in context_words:
        index_of_word_dictionary = word_to_index.get(word) 
        ctxt_word_vector[index_of_word_dictionary] = 1
        
    return trgt_word_vector,ctxt_word_vector

In [7]:
def get_data(corpus,length_of_corpus,window_size,method):
    training_data =  []
    for i,word in enumerate(corpus):
        
        index_target_word = i
        target_word = word
        context_words = []
        
        if i == 0:  
            context_words = [corpus[x] for x in range(i + 1 , window_size + 1)] 

        elif i == len(corpus)-1:
            context_words = [corpus[x] for x in range(length_of_corpus - 2 ,length_of_corpus -2 - window_size  , -1 )]

        else:
            before_target_word_index = index_target_word - 1
            for x in range(before_target_word_index, before_target_word_index - window_size , -1):
                if x >=0:
                    context_words.extend([corpus[x]])

            after_target_word_index = index_target_word + 1
            for x in range(after_target_word_index, after_target_word_index + window_size):
                if x < len(corpus):
                    context_words.extend([corpus[x]])
        if method == 'single':
            trgt_word_vector,ctxt_word_vector = get_one_hot_single_vectors(target_word,context_words)
        elif method == 'multiple':
            trgt_word_vector,ctxt_word_vector = get_one_hot_multiple_vectors(target_word,context_words)
        
        training_data.append([trgt_word_vector,ctxt_word_vector])
    return training_data



In [53]:
word_to_index,index_to_word,corpus,length_of_corpus,vocab_size = get_inital_values(text)

In [54]:
print('Number of unique words:' , vocab_size)
#print('word_to_index : ',word_to_index)
#print('index_to_word : ',index_to_word)
#print('corpus:',corpus)
print('Length of corpus :',length_of_corpus)
    

Number of unique words: 449
Length of corpus : 845


In [55]:
window_size = 5

<h4> Method 1 : multiple arrays for context word </h4>

In [56]:
training_data = get_data(corpus,length_of_corpus,window_size,'multiple')

In [57]:
print('Size : %.4f kbs' %(asizeof(training_data)/1024))

Size : 33599.3203 kbs


<h4> Method 2 : single arrays for context word </h4>

In [58]:
training_data = get_data(corpus,length_of_corpus,window_size,'single')

In [59]:
print('Size : %.4f kbs' %(asizeof(training_data)/1024))

Size : 6159.5469 kbs
